In [117]:
# %run feature_generator/spectral.ipynb
# %run feature_generator/sample_entropy.ipynb
# %run feature_generator/spectral_entropy.ipynb
%run feature_generator/stats.ipynb
# %run feature_generator/hjorth.ipynb
# %run feature_generator/entropies.ipynb
%run load_labels.ipynb

In [118]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import keras
from keras.models import load_model
from keras import backend as K
from keras import models
from keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.utils import to_categorical


In [119]:
num_classes = 2
channels = 32
sfreq = 128

In [122]:
data = features

In [123]:
labels = load_labels()
label = pd.concat([labels['t1_math'], labels['t2_math'],
                  labels['t3_math']]).to_numpy()
label = label.repeat(data.shape[0]//label.shape[0])

# Linear Regression

In [124]:
x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
lr_clf = LogisticRegression(max_iter=1000).fit(x_train, y_train)
y_pred = lr_clf.predict(x_test)
y_true = y_test


In [125]:
scores_lr = lr_clf.score(x_test, y_test)
precision_lr = metrics.precision_score(y_true, y_pred, average='macro')
recall_lr = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_lr = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_lr)
print('precision is:', precision_lr)
print('recall is:', recall_lr)
print('f1_score is:', f1_score_lr)
metrics.confusion_matrix(y_true, y_pred)


accuracy is: 0.592929292929293
precision is: 0.5917230890000982
recall is: 0.592929292929293
f1_score is: 0.5931721297324787


array([[320, 207],
       [196, 267]])

# KNN Classifier

In [126]:
K.clear_session()
x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
neigh_clf = KNeighborsClassifier(n_neighbors=2)
neigh_clf.fit(x_train, y_train)
y_pred = neigh_clf.predict(x_test)
y_true = y_test


In [127]:
scores_neigh = neigh_clf.score(x_test, y_test)
precision_neigh = metrics.precision_score(y_true, y_pred, average='macro')
recall_neigh = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_neigh = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_neigh)
print('precision is:', precision_neigh)
print('recall is:', recall_neigh)
print('f1_score is:', f1_score_neigh)
metrics.confusion_matrix(y_true, y_pred)


accuracy is: 0.5262626262626262
precision is: 0.5132894563354283
recall is: 0.5262626262626262
f1_score is: 0.49363989030482


array([[402, 125],
       [344, 119]])

# SVM Classifier

In [128]:
x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
svm_clf = SVC()
svm_clf.fit(x_train, y_train)
y_pred = svm_clf.predict(x_test)
y_true = y_test


In [129]:
scores_svm = svm_clf.score(x_test, y_test)
precision_svm = metrics.precision_score(y_true, y_pred, average='macro')
recall_svm = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_svm = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_svm)
print('precision is:', precision_svm)
print('recall is:', recall_svm)
print('f1_score is:', f1_score_svm)
metrics.confusion_matrix(y_true, y_pred)


accuracy is: 0.5080808080808081
precision is: 0.5131333021827649
recall is: 0.5080808080808081
f1_score is: 0.505916385557112


array([[231, 296],
       [191, 272]])

# Neural Network

In [130]:
K.clear_session()
y_v = label
y_v = to_categorical(y_v, num_classes)
x_train, x_test, y_train, y_test = train_test_split(
    data, y_v, test_size=0.33, random_state=42)

In [131]:
x_in = keras.Input(shape=(x_train.shape[1],))
x = Dense(500, kernel_initializer='normal', activation='relu')(x_in)
x = Dense(300, kernel_initializer='normal', activation='relu')(x)
x = Dense(200, kernel_initializer='normal', activation='relu')(x)
x = Dense(50, kernel_initializer='normal', activation='relu')(x)
out_layer = Dense(num_classes, activation='softmax', name='out')(x)
model = keras.Model(x_in, out_layer)
#         model.summary()
# Compile model
#         loss_fn = keras.losses.SparseCategoricalCrossentropy()
#         loss_fn = keras.losses.categorical_crossentropy()

In [132]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
#model.summary()
# Fit the model
model.fit(x_train, y_train, epochs=100, batch_size=128, verbose=0)
# evaluate the model
scores_dnn = model.evaluate(x_test, y_test, verbose=0)

2022-10-18 12:09:00.413199: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-18 12:09:14.565140: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [133]:
y_pred = model.predict(x_test)
y_true = y_test
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_true, axis=1)

31/31 [==============================] - 0s 3ms/step


2022-10-18 12:09:19.405408: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [134]:
precision_dnn = metrics.precision_score(y_true, y_pred, average='macro')
recall_dnn = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_dnn = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_dnn[1])
print('precision is:', precision_dnn)
print('recall is:', recall_dnn)
print('f1_score is:', f1_score_dnn)
metrics.confusion_matrix(y_true, y_pred)

accuracy is: 0.649495005607605
precision is: 0.6509580485890855
recall is: 0.6494949494949495
f1_score is: 0.6497320935882428


array([[328, 199],
       [148, 315]])